In [ ]:
import pandas as pd
import numpy as np
import requests
import math
from math import sqrt

# -----------------------------
# 0) Utilities
# -----------------------------
def haversine(lat1, lon1, lat2, lon2):
    """
    lat1, lon1: scalar
    lat2, lon2: numpy arrays
    return: numpy array distances (km)
    """
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def is_wgs84_like(lat_series, lon_series):
    lat_ok = lat_series.between(-90, 90).mean()
    lon_ok = lon_series.between(-180, 180).mean()
    return (lat_ok > 0.99) and (lon_ok > 0.99)

def get_elevations_batched(locations, batch_size=100):
    """
    Open-Elevation 批次查詢；避免一次塞太多 locations 造成失敗。
    locations: list of (lat, lon)
    """
    out = []
    for i in range(0, len(locations), batch_size):
        chunk = locations[i:i + batch_size]
        loc_str = "|".join([f"{lat},{lon}" for lat, lon in chunk])
        url = f"https://api.open-elevation.com/api/v1/lookup?locations={loc_str}"
        try:
            resp = requests.get(url, timeout=20)
            if resp.status_code == 200:
                data = resp.json()
                if "results" in data and len(data["results"]) == len(chunk):
                    out.extend([r.get("elevation", np.nan) for r in data["results"]])
                else:
                    out.extend([np.nan] * len(chunk))
            else:
                out.extend([np.nan] * len(chunk))
        except Exception as e:
            print(f"API error: {e}")
            out.extend([np.nan] * len(chunk))
    return out

def get_season(date_str):
    # 南半球: 1=夏季 Dec-Feb, 2=秋季 Mar-May, 3=冬季 Jun-Aug, 4=春季 Sep-Nov
    try:
        month = pd.to_datetime(date_str, format="%d-%m-%Y").month
        if month in [12, 1, 2]: return 1
        elif month in [3, 4, 5]: return 2
        elif month in [6, 7, 8]: return 3
        else: return 4
    except:
        return np.nan

# -----------------------------
# 1) Core Pipeline
# -----------------------------
def build_features(
    water_df: pd.DataFrame,
    dams_df: pd.DataFrame,
    radii_km=(5, 10, 25),
    in_dam_km=5.0,
    use_elevation=True,
    elevation_batch_size=100,
):
    """
    將 water_df 加上 dam-based features。
    所有 feature 均為「相對量」或「周邊統計摘要」，不暴露特定 dam 的 ID 或絕對位置。
    """

    # ---- copy to avoid in-place side effects
    water_df = water_df.copy()
    dams_df  = dams_df.copy()

    # ---- rename columns to standard
    if "Y_COORD" in dams_df.columns:
        dams_df = dams_df.rename(columns={"Y_COORD": "dam_lat", "X_COORD": "dam_lon"})
    if "Latitude" in water_df.columns:
        water_df = water_df.rename(columns={"Latitude": "sample_lat", "Longitude": "sample_lon"})

    # ---- sanity check
    if not is_wgs84_like(dams_df["dam_lat"], dams_df["dam_lon"]):
        print("WARNING: dam_lat/dam_lon 看起來不像 WGS84 經緯度")
    if not is_wgs84_like(water_df["sample_lat"], water_df["sample_lon"]):
        print("WARNING: sample_lat/sample_lon 看起來不像 WGS84 經緯度")

    # ---- precompute dam arrays
    dam_lats = dams_df["dam_lat"].to_numpy()
    dam_lons = dams_df["dam_lon"].to_numpy()

    has_area    = "Shape_Area"   in dams_df.columns
    has_length  = "Shape_Length" in dams_df.columns
    has_area_km = "AREA_KM2"     in dams_df.columns

    dam_shape_area   = dams_df["Shape_Area"].to_numpy()   if has_area    else np.zeros(len(dams_df))
    dam_shape_length = dams_df["Shape_Length"].to_numpy() if has_length  else np.zeros(len(dams_df))
    dam_area_km2     = dams_df["AREA_KM2"].to_numpy()     if has_area_km else np.zeros(len(dams_df))

    # pre-compute per-dam geometry ratios (shape-level, not location-level)
    eps = 1e-12
    dam_apr   = dam_shape_area / (dam_shape_length + eps)                         # area/perimeter ratio
    dam_comp  = (4.0 * math.pi * dam_shape_area) / ((dam_shape_length + eps)**2)  # compactness

    # ---- per-sample feature accumulators
    dist_nearest    = []
    within_counts   = {r: [] for r in radii_km}
    within_area_sum = {r: [] for r in radii_km}
    within_area_mean= {r: [] for r in radii_km}
    within_area_max = {r: [] for r in radii_km}
    within_apr_mean = {r: [] for r in radii_km}   # mean area/perimeter ratio
    within_comp_mean= {r: [] for r in radii_km}   # mean compactness
    within_dist_mean= {r: [] for r in radii_km}   # mean distance to dams inside radius
    nearest_apr     = []
    nearest_comp    = []
    nearest_area_km = []
    nearest_area_norm = []   # nearest dam area / mean area of all dams in same radius band

    global_mean_area = np.nanmean(dam_shape_area) if has_area else np.nan

    # elevation list for sample points (filled if use_elevation)
    sample_elev_list = [np.nan] * len(water_df)
    # we also need nearest-dam elevations for relative elevation feature
    nearest_dam_lat_list = []
    nearest_dam_lon_list = []

    for i, (_, row) in enumerate(water_df.iterrows()):
        dists = haversine(row["sample_lat"], row["sample_lon"], dam_lats, dam_lons)
        idx   = int(np.argmin(dists))

        dist_nearest.append(float(dists[idx]))
        nearest_dam_lat_list.append(float(dam_lats[idx]))
        nearest_dam_lon_list.append(float(dam_lons[idx]))

        # nearest dam geometry (shape descriptors, not raw area/coords)
        nearest_apr.append(float(dam_apr[idx]))
        nearest_comp.append(float(dam_comp[idx]))
        nearest_area_km.append(float(dam_area_km2[idx]) if has_area_km else np.nan)

        for r in radii_km:
            mask = dists <= r
            n    = int(mask.sum())
            within_counts[r].append(n)

            areas_r  = dam_shape_area[mask]
            dists_r  = dists[mask]
            apr_r    = dam_apr[mask]
            comp_r   = dam_comp[mask]

            within_area_sum[r].append(float(areas_r.sum())  if n > 0 else 0.0)
            within_area_mean[r].append(float(areas_r.mean()) if n > 0 else np.nan)
            within_area_max[r].append(float(areas_r.max())  if n > 0 else np.nan)
            within_apr_mean[r].append(float(apr_r.mean())   if n > 0 else np.nan)
            within_comp_mean[r].append(float(comp_r.mean()) if n > 0 else np.nan)
            within_dist_mean[r].append(float(dists_r.mean()) if n > 0 else np.nan)

        # nearest dam area normalised by global mean (removes absolute scale)
        nearest_area_norm.append(
            float(dam_shape_area[idx] / global_mean_area)
            if (has_area and global_mean_area > 0) else np.nan
        )

    # ---- store non-location features
    water_df["distance_to_nearest_dam_km"] = dist_nearest

    # log-distance is more generalizable than raw distance
    water_df["log1p_distance_to_nearest_dam"] = np.log1p(water_df["distance_to_nearest_dam_km"])

    # is sample point "inside" nearest dam footprint?
    water_df["is_in_dam_zone"] = (water_df["distance_to_nearest_dam_km"] < in_dam_km).astype(int)

    # nearest dam shape descriptors
    water_df["nearest_dam_area_perimeter_ratio"] = nearest_apr
    water_df["nearest_dam_compactness"]           = nearest_comp
    water_df["nearest_dam_area_km2"]              = nearest_area_km
    water_df["nearest_dam_area_norm"]             = nearest_area_norm  # relative to global mean

    # per-radius aggregated features
    for r in radii_km:
        water_df[f"dam_count_within_{r}km"]          = within_counts[r]
        water_df[f"dam_area_sum_within_{r}km"]        = within_area_sum[r]
        water_df[f"dam_area_mean_within_{r}km"]       = within_area_mean[r]
        water_df[f"dam_area_max_within_{r}km"]        = within_area_max[r]
        water_df[f"dam_apr_mean_within_{r}km"]        = within_apr_mean[r]
        water_df[f"dam_compactness_mean_within_{r}km"]= within_comp_mean[r]
        water_df[f"dam_dist_mean_within_{r}km"]       = within_dist_mean[r]

    # dam density proxies (count / area of search circle)
    for r in radii_km:
        circle_area = math.pi * r**2  # km²
        water_df[f"dam_density_{r}km"] = water_df[f"dam_count_within_{r}km"] / circle_area

    # multi-scale dam pressure: weighted sum of counts by inverse-radius
    if len(radii_km) >= 2:
        water_df["dam_pressure_score"] = sum(
            water_df[f"dam_count_within_{r}km"] / r for r in radii_km
        )

    # ---- elevation features (relative only — no absolute values retained)
    if use_elevation:
        sample_locations = list(zip(water_df["sample_lat"], water_df["sample_lon"]))
        dam_locations    = list(zip(nearest_dam_lat_list, nearest_dam_lon_list))

        all_locations = sample_locations + dam_locations
        all_elev      = get_elevations_batched(all_locations, batch_size=elevation_batch_size)

        sample_elevs = np.array(all_elev[:len(sample_locations)], dtype=float)
        nearest_dam_elevs = np.array(all_elev[len(sample_locations):], dtype=float)

        # RELATIVE elevation: how much higher/lower is sample vs nearest dam
        elev_diff = sample_elevs - nearest_dam_elevs
        water_df["elev_diff_sample_minus_dam"] = elev_diff  # positive => upstream, negative => downstream

        # categorical position (still useful as a feature, but derived from relative not absolute elevation)
        def _pos(diff, dist, in_dam_km):
            if np.isnan(diff): return "unknown"
            if dist < in_dam_km: return "in_dam"
            if diff > 0:  return "upstream"
            if diff < 0:  return "downstream"
            return "in_dam"

        water_df["position_relative_to_dam"] = [
            _pos(d, dist, in_dam_km)
            for d, dist in zip(elev_diff, dist_nearest)
        ]
    else:
        water_df["elev_diff_sample_minus_dam"]  = np.nan
        water_df["position_relative_to_dam"]    = "unknown"

    # ---- season + evap proxy (time-based, not location-based)
    if "Sample Date" in water_df.columns:
        water_df["season"] = water_df["Sample Date"].apply(get_season)
    else:
        water_df["season"] = np.nan

    # evap proxy uses nearest dam area (already captured above) × season factor
    water_df["evap_proxy"] = (
        water_df["nearest_dam_area_km2"]
        * water_df["season"].apply(lambda s: 1.5 if s == 1 else 1.0)
    )

    # ---- ALWAYS drop raw coordinates and any ID/name columns
    cols_to_drop = [
        "sample_lat", "sample_lon",   # raw location — drop to prevent leakage
        "nearest_dam_id",              # identity of dam leaks location
        "dam_lat", "dam_lon",
        "NAME", "STATION", "ALIAS", "TYPE",
        "OBJECTID", "FEAT_ID", "NEAREST_DAM_ID",
        "DAMS500G_", "DAMS500G_I",
        "has_station_code", "has_alias",
        # raw single-dam geometry (replaced by shape-descriptor + normalised versions)
        "Shape_Area", "Shape_Length", "AREA", "PERIMETER", "AREA_KM2",
        # absolute elevation values
        "sample_elevation", "dam_elevation",
    ]
    water_df = water_df.drop(columns=cols_to_drop, errors="ignore")

    return water_df

# -----------------------------
# 2) Runner: apply to train & valid separately
# -----------------------------
def run_pipeline_for_two_sets(
    dams_path,
    train_path,
    valid_path,
    out_train_path="enhanced_train.csv",
    out_valid_path="enhanced_valid.csv",
    use_elevation=True
):
    dams_df  = pd.read_csv(dams_path)
    train_df = pd.read_csv(train_path)
    valid_df = pd.read_csv(valid_path)

    train_enh = build_features(train_df, dams_df, use_elevation=use_elevation)
    valid_enh = build_features(valid_df, dams_df, use_elevation=use_elevation)

    train_enh.to_csv(out_train_path, index=False)
    valid_enh.to_csv(out_valid_path, index=False)

    print("Train enhanced shape:", train_enh.shape)
    print("Valid enhanced shape:",  valid_enh.shape)
    print("\nNew dam-related columns:")
    dam_cols = [c for c in train_enh.columns if any(
        kw in c for kw in ["dam", "elev", "position", "evap", "season", "compactness", "density", "pressure"]
    )]
    for c in dam_cols:
        print(" ", c)

    return train_enh, valid_enh

# -----------------------------
# 3) Example usage
# -----------------------------
if __name__ == "__main__":
    dams_path  = "Dams_dataset.csv"
    train_path = "water_quality_training_dataset.csv"
    valid_path = "submission_template.csv"

    train_enh, valid_enh = run_pipeline_for_two_sets(
        dams_path=dams_path,
        train_path=train_path,
        valid_path=valid_path,
        out_train_path="dams_training_dataset.csv",
        out_valid_path="dams_validation_dataset.csv",
        use_elevation=True  # set False to skip Open-Elevation API calls
    )


## 修改說明：如何降低位置洩漏

### 原版問題
| 原始欄位 | 洩漏原因 |
|---|---|
| `nearest_dam_id` | 直接 encode 哪個 dam，等於暴露位置 |
| `NAME`, `STATION`, `ALIAS` | 特定 dam 識別符 |
| `TYPE` | 結合距離可反推位置 |
| `AREA`, `Shape_Area`, `Shape_Length` (raw) | 某一特定 dam 的值，模型可記憶 |
| `sample_lat`, `sample_lon` | 直接暴露位置 |
| `sample_elevation`, `dam_elevation` (絕對值) | 絕對海拔含地理資訊 |

### 修改策略
1. **移除所有 ID 和識別欄位**：`nearest_dam_id`, `NAME`, `STATION`, `ALIAS`, `OBJECTID` 等全部 drop。
2. **以相對量取代絕對量**：絕對海拔 → `elev_diff_sample_minus_dam`（相對高差），模型學到的是「sample 比 dam 高多少」而非「在哪裡」。
3. **以形狀描述子取代原始幾何數值**：raw `Shape_Area` / `Shape_Length` → `compactness`、`area_perimeter_ratio`，加上 `nearest_dam_area_norm`（相對全域均值的比例），模型看到的是「這個 dam 的形狀特性」而非「某個特定 dam 的尺寸」。
4. **以多半徑統計摘要取代單點查找**：`dam_count`, `dam_area_mean`, `dam_area_max`, `dam_apr_mean`, `dam_compactness_mean`, `dam_dist_mean` 等都是「周邊 dam 群的統計」，不指向任何特定 dam。
5. **dam density 和 dam pressure**：新增密度（count/面積）和加權壓力分數，是 positional-agnostic 的環境指標。
6. **drop 原始座標**：`sample_lat`, `sample_lon` 也一併 drop，避免模型透過座標直接記憶訓練點。
